In [10]:
import warnings
warnings.filterwarnings('ignore')
from product_details import Parameters
pm = Parameters()
from greedyConstructive import dc_greedy_constructive_search

def calculate_costs(dc, as_is_dc=False):
    all_costs = pm.total_costs(dc)

    if as_is_dc:
        all_costs = pm.total_costs(dc, as_is_dc)
        
    print('Total Obj. Value:', round(all_costs[0]))
    print('Total Costs:', round(all_costs[1]))
    print('Total Inbound Costs:', round(all_costs[2]))
    print('Total Handling In Costs:', round(all_costs[3]))
    print('Total Storage Costs:', round(all_costs[4]))
    print('Total Handling Out Costs:', round(all_costs[5]))
    print('Total Outbound Costs:', round(all_costs[6]))
    print('Total Operational Costs:', round(all_costs[7]))
    print('Total Gas Emissions (Inbound):', round(all_costs[8]))


In [11]:
# As-Is Situation

# Define the as-is DC allocation. The keys are the DC, the values are the states where the corresponding DC's customers are.
as_is_dc = {
    "NY": ['CT', 'DC', 'DE', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT'],
    "ND": ['MN', 'MT', 'ND', 'SD', 'WY'],
    "IL": ['HI', 'IA', 'IL', 'IN', 'KS', 'KY', 'MI', 'MO', 'NE', 'OH', 'VA', 'WI', 'WV'],
    "TN": ['AL', 'FL', 'GA', 'NC', 'SC', 'TN'],
    "WA": ['AK', 'ID', 'OR', 'WA'],
    "TX": ['AR', 'AZ', 'LA', 'MS', 'NM', 'OK', 'TX'],
    "UT": ['CA', 'CO', 'NV', 'UT']
}

calculate_costs(as_is_dc)

Total Obj. Value: 12913202
Total Costs: 11363602
Total Inbound Costs: 1124750
Total Handling In Costs: 55780
Total Storage Costs: 1341612
Total Handling Out Costs: 686104
Total Outbound Costs: 1155356
Total Operational Costs: 7000000
Total Gas Emissions (Inbound): 1549600


In [13]:
# Algorithm 1: Greedy Constructive Search
# Allocate dc based on the distance/costs greedily 
greedy_dc_allocation = dc_greedy_constructive_search(open_dc=True)

calculate_costs(greedy_dc_allocation, as_is_dc)
# Turns out, even though the shipping costs are lower,
# the total outbound costs are relatively higher since the operating and opening costs are skyrocketed.

Total Obj. Value: 17138120
Total Costs: 15615620
Total Inbound Costs: 1215700
Total Handling In Costs: 60340
Total Storage Costs: 1278576
Total Handling Out Costs: 697577
Total Outbound Costs: 933427
Total Operational Costs: 11430000
Total Gas Emissions (Inbound): 1522500


In [21]:
# Algorithm 2: Greedy Constructive Heuristics
# Now, knowing that opening and operating new DC can be costly, we apply the same algorithm without opening a new DC.
greedy_dc_allocation = dc_greedy_constructive_search(open_dc=False)
calculate_costs(greedy_dc_allocation, as_is_dc)
print(greedy_dc_allocation)
# The shipping costs are lowered again, but the handling costs are increasing slightly.
# It resulted in a less significant decrease.

Total Obj. Value: 12535644
Total Costs: 10959444
Total Inbound Costs: 1128000
Total Handling In Costs: 47640
Total Storage Costs: 1116156
Total Handling Out Costs: 710385
Total Outbound Costs: 957264
Total Operational Costs: 7000000
Total Gas Emissions (Inbound): 1576200
{'WA': ['AK', 'ID', 'OR', 'WA'], 'TN': ['AL', 'AR', 'FL', 'GA', 'KS', 'KY', 'MO', 'MS', 'NC', 'OH', 'PA', 'SC', 'TN', 'VA', 'WV'], 'UT': ['AZ', 'CA', 'CO', 'NV', 'UT', 'WY'], 'NY': ['CT', 'DE', 'HI', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT', 'DC'], 'ND': ['IA', 'IN', 'MN', 'MT', 'ND', 'NE', 'SD', 'WI'], 'IL': ['IL', 'MI'], 'TX': ['LA', 'NM', 'OK', 'TX']}


In [20]:
# Algorithm 3: Greedy Constructive Heuristics
# Closing certain DCs that has minimal closing costs.

greedy_dc_allocation = {}
outbound_data = pm.outbound_data
as_is = list(as_is_dc.keys())
as_is.remove('WA')
as_is.remove('ND')

for state in outbound_data.index:
    current_dc = outbound_data[as_is]
    best_dc = (current_dc.loc[state]).idxmin()
    
    # if best_dc not in as_is:
    #     print(best_dc)
    
    if best_dc not in greedy_dc_allocation:
        greedy_dc_allocation[best_dc] = [state]
    elif isinstance(greedy_dc_allocation[best_dc], list):
        greedy_dc_allocation[best_dc].append(state)
    else:
        greedy_dc_allocation[best_dc] = [greedy_dc_allocation[best_dc], state]

calculate_costs(greedy_dc_allocation, as_is_dc)
print(greedy_dc_allocation)

Total Obj. Value: 11649452
Total Costs: 10073652
Total Inbound Costs: 1107050
Total Handling In Costs: 48950
Total Storage Costs: 1157412
Total Handling Out Costs: 710973
Total Outbound Costs: 1024267
Total Operational Costs: 6025000
Total Gas Emissions (Inbound): 1575800
{'UT': ['AK', 'AZ', 'CA', 'CO', 'ID', 'MT', 'NV', 'OR', 'UT', 'WA', 'WY'], 'TN': ['AL', 'AR', 'FL', 'GA', 'KS', 'KY', 'MO', 'MS', 'NC', 'NE', 'OH', 'PA', 'SC', 'TN', 'VA', 'WV'], 'NY': ['CT', 'DE', 'HI', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT', 'DC'], 'IL': ['IA', 'IL', 'IN', 'MI', 'MN', 'ND', 'SD', 'WI'], 'TX': ['LA', 'NM', 'OK', 'TX']}


In [22]:
# Best solution (Simulated Annealing)
to_be = {'WA': ['AK', 'MT', 'WA'], 'TN': ['IN', 'MI', 'MS', 'OH', 'VA', 'MA', 'TN', 'PA', 'NJ', 'NH', 'KS', 'KY', 'DE', 'GA', 'AR', 'MO', 'MD', 'AL', 'NC', 'IL', 'UT', 'SC', 'WI', 'WV', 'CT', 'IA', 'ND', 'MN', 'SD', 'FL'], 'TX': ['TX', 'LA', 'OK'], 'UT': ['NM', 'AZ', 'OR', 'NE', 'NV', 'HI', 'CA', 'ID', 'CO', 'WY'], 'CA': [], 'NY': ['NY', 'RI'], 'PA': [], 'KS': [], 'IL': ['DC', 'VT', 'ME'], 'ND': []}
calculate_costs(to_be, as_is_dc)

# In terms of costs, the solution from Greedy Constructive Search is more efficient.
# But the total gas emission is better with Simulated Annealing.

Total Obj. Value: 11858602
Total Costs: 10330102
Total Inbound Costs: 1146000
Total Handling In Costs: 43630
Total Storage Costs: 828036
Total Handling Out Costs: 735225
Total Outbound Costs: 1152211
Total Operational Costs: 6425000
Total Gas Emissions (Inbound): 1528500


In [24]:
# Best solution (Tabu Search)
dc = {'WA': ['AK', 'ID', 'OR', 'WA'], 'ND': ['IA', 'IN', 'MN', 'MT', 'ND', 'NE', 'SD', 'WI'], 'IL': ['IL', 'MI'], 'TX': ['LA', 'NM', 'OK', 'TX'], 'TN': ['AL', 'AR', 'FL', 'GA', 'KS', 'KY', 'MO', 'MS', 'NC', 'OH', 'PA', 'SC', 'TN', 'VA', 'WV'], 'UT': ['AZ', 'CA', 'CO', 'NV', 'UT', 'WY'], 'NY': ['CT', 'DE', 'HI', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT', 'DC']}
calculate_costs(dc, as_is_dc)

Total Obj. Value: 12535644
Total Costs: 10959444
Total Inbound Costs: 1128000
Total Handling In Costs: 47640
Total Storage Costs: 1116156
Total Handling Out Costs: 710385
Total Outbound Costs: 957264
Total Operational Costs: 7000000
Total Gas Emissions (Inbound): 1576200
